<a href="https://colab.research.google.com/github/bobbercheng/ventilator-pressure-prediction/blob/master/ventilator_pressure_1d_cnn_2048_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pandas==1.3.2

In [2]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [3]:
!pip install kaggle
!mkdir /root/.kaggle
!cp /gdrive/MyDrive/ventilator-pressure-prediction/kaggle.json /root/.kaggle
!kaggle competitions download -c ventilator-pressure-prediction
!mkdir -p /kaggle/input/ventilator-pressure-prediction
!unzip '*.zip' -d /kaggle/input/ventilator-pressure-prediction
!ls /kaggle/input/ventilator-pressure-prediction

  0% 0.00/8.50M [00:00<?, ?B/s]
100% 8.50M/8.50M [00:00<00:00, 78.5MB/s]
 93% 130M/139M [00:00<00:00, 140MB/s]
100% 139M/139M [00:01<00:00, 137MB/s]
 97% 73.0M/75.4M [00:00<00:00, 174MB/s]
100% 75.4M/75.4M [00:00<00:00, 170MB/s]
Archive:  train.csv.zip
  inflating: /kaggle/input/ventilator-pressure-prediction/train.csv  

Archive:  sample_submission.csv.zip
  inflating: /kaggle/input/ventilator-pressure-prediction/sample_submission.csv  

Archive:  test.csv.zip
  inflating: /kaggle/input/ventilator-pressure-prediction/test.csv  

3 archives were successfully processed.
sample_submission.csv  test.csv  train.csv


In [4]:
import numpy as np
import pandas as pd

# import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

In [5]:
DEBUG = False

train = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('/kaggle/input/ventilator-pressure-prediction/sample_submission.csv')

if DEBUG:
    train = train[:80*1000]

In [6]:
def add_feats(train):
    # # rewritten calculation of lag features from this notebook: https://www.kaggle.com/patrick0302/add-lag-u-in-as-new-feat
# # some of ideas from this notebook: https://www.kaggle.com/mst8823/google-brain-lightgbm-baseline
    # train[["15_out_mean"]] = train.groupby('breath_id')['u_out'].rolling(window=15,min_periods=1).agg({"15_out_mean":"mean"}).reset_index(level=0,drop=True)
    train['last_value_u_in'] = train.groupby('breath_id')['u_in'].transform('last')
    train['u_in_lag1'] = train.groupby('breath_id')['u_in'].shift(1)
    train['u_out_lag1'] = train.groupby('breath_id')['u_out'].shift(1)
    train['u_in_lag_back1'] = train.groupby('breath_id')['u_in'].shift(-1)
    train['u_out_lag_back1'] = train.groupby('breath_id')['u_out'].shift(-1)
    train['u_in_lag2'] = train.groupby('breath_id')['u_in'].shift(2)
    train['u_out_lag2'] = train.groupby('breath_id')['u_out'].shift(2)
    train['u_in_lag3'] = train.groupby('breath_id')['u_in'].shift(3)
    train['u_out_lag3'] = train.groupby('breath_id')['u_out'].shift(3)
    train['u_in_lag_back2'] = train.groupby('breath_id')['u_in'].shift(-2)
    train['u_out_lag_back2'] = train.groupby('breath_id')['u_out'].shift(-2)
    train['u_in_lag_back3'] = train.groupby('breath_id')['u_in'].shift(-3)
    train['u_out_lag_back3'] = train.groupby('breath_id')['u_out'].shift(-3)
    train['u_in_lag_back10'] = train.groupby('breath_id')['u_in'].shift(-10)
    train['u_out_lag_back10'] = train.groupby('breath_id')['u_out'].shift(-10)

    train['u_in_first'] = train.groupby('breath_id')['u_in'].first()
    train['u_out_first'] = train.groupby('breath_id')['u_out'].first()

    ## time since last step
    train['time_step_diff'] = train.groupby('breath_id')['time_step'].diff().fillna(0)
    ### rolling window ts feats
    train['ewm_u_in_mean'] = train.groupby('breath_id')['u_in'].ewm(halflife=9).mean().reset_index(level=0,drop=True)
    train['ewm_u_in_std'] = train.groupby('breath_id')['u_in'].ewm(halflife=10).std().reset_index(level=0,drop=True) ## could add covar?
    train['ewm_u_in_corr'] = train.groupby('breath_id')['u_in'].ewm(halflife=15).corr().reset_index(level=0,drop=True) # self umin corr
    # train['ewm_u_in_corr'] = train.groupby('breath_id')['u_in'].ewm(halflife=6).corr(train.groupby('breath_id')["u_out"]).reset_index(level=0,drop=True) # corr with u_out # error
    ## rolling window of 15 periods
    train[["15_in_sum","15_in_min","15_in_max","15_in_mean","15_out_std"]] = train.groupby('breath_id')['u_in'].rolling(window=15,min_periods=1).agg({"15_in_sum":"sum","15_in_min":"min","15_in_max":"max","15_in_mean":"mean","15_in_std":"std"}).reset_index(level=0,drop=True)
#     train[["45_in_sum","45_in_min","45_in_max","45_in_mean","45_out_std"]] = train.groupby('breath_id')['u_in'].rolling(window=45,min_periods=1).agg({"45_in_sum":"sum","45_in_min":"min","45_in_max":"max","45_in_mean":"mean","45_in_std":"std"}).reset_index(level=0,drop=True)
    train[["45_in_sum","45_in_min","45_in_max","45_in_mean","45_out_std"]] = train.groupby('breath_id')['u_in'].rolling(window=45,min_periods=1).agg({"45_in_sum":"sum","45_in_min":"min","45_in_max":"max","45_in_mean":"mean","45_in_std":"std"}).reset_index(level=0,drop=True)

    train[["15_out_mean"]] = train.groupby('breath_id')['u_out'].rolling(window=15,min_periods=1).agg({"15_out_mean":"mean"}).reset_index(level=0,drop=True)

    print(train.shape[0])
    display(train)
    train = train.fillna(0) # ORIG

    # max, min, mean value of u_in and u_out for each breath
    train['breath_id__u_in__max'] = train.groupby(['breath_id'])['u_in'].transform('max')
    train['breath_id__u_out__max'] = train.groupby(['breath_id'])['u_out'].transform('max')

    train['breath_id__u_out__mean'] =train.groupby(['breath_id'])['u_out'].mean()
    train['breath_id__u_in__mean'] =train.groupby(['breath_id'])['u_in'].mean()

    train['breath_id__u_in__min'] = train.groupby(['breath_id'])['u_in'].transform('min')
    train['breath_id__u_out__min'] = train.groupby(['breath_id'])['u_out'].transform('min')

    train['u_in_first'] = train.groupby(['breath_id'])['u_in'].transform('first')
    train['u_in_last'] = train.groupby(['breath_id'])['u_in'].transform('last')

    train['R_div_C'] = train["R"].div(train["C"])

    # difference between consequitive values
    train['R__C'] = train["R"].astype(str) + '__' + train["C"].astype(str)
    train['u_in_diff1'] = train['u_in'] - train['u_in_lag1']
    train['u_out_diff1'] = train['u_out'] - train['u_out_lag1']
    train['u_in_diff2'] = train['u_in'] - train['u_in_lag2']
    train['u_out_diff2'] = train['u_out'] - train['u_out_lag2']
    train['u_in_diff3'] = train['u_in'] - train['u_in_lag3']
    train['u_out_diff3'] = train['u_out'] - train['u_out_lag3']
    ## diff between last 2 steps
    train['u_in_diff_1_2'] = train['u_in_lag1'] - train['u_in_lag2']
    train['u_out_diff_1_2'] = train['u_out_lag1'] - train['u_out_lag2']
    train['u_in_lagback_diff_1_2'] = train['u_in_lag_back1'] - train['u_in_lag_back2']
    train['u_out_lagback_diff_1_2'] = train['u_out_lag_back1'] - train['u_out_lag_back2']

    train['u_in_lagback_diff1'] = train['u_in'] - train['u_in_lag_back1']
    train['u_out_lagback_diff1'] = train['u_out'] - train['u_out_lag_back1']
    train['u_in_lagback_diff2'] = train['u_in'] - train['u_in_lag_back2']
    train['u_out_lagback_diff2'] = train['u_out'] - train['u_out_lag_back2']

    # from here: https://www.kaggle.com/yasufuminakama/ventilator-pressure-lstm-starter
    train.loc[train['time_step'] == 0, 'u_in_diff'] = 0
    train.loc[train['time_step'] == 0, 'u_out_diff'] = 0

    # difference between the current value of u_in and the max value within the breath
    train['breath_id__u_in__diffmax'] = train.groupby(['breath_id'])['u_in'].transform('max') - train['u_in']
    train['breath_id__u_in__diffmean'] = train.groupby(['breath_id'])['u_in'].transform('mean') - train['u_in']

    print("before OHE")
    display(train)

    # OHE
    train = train.merge(pd.get_dummies(train['R'], prefix='R'), left_index=True, right_index=True).drop(['R'], axis=1)
    train = train.merge(pd.get_dummies(train['C'], prefix='C'), left_index=True, right_index=True).drop(['C'], axis=1)
    train = train.merge(pd.get_dummies(train['R__C'], prefix='R__C'), left_index=True, right_index=True).drop(['R__C'], axis=1)

    # https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/273974
    train['u_in_cumsum'] = train.groupby(['breath_id'])['u_in'].cumsum()
    train['time_step_cumsum'] = train.groupby(['breath_id'])['time_step'].cumsum()

    # feature by u in or out (ideally - make 2 sep columns for each state) # dan
    train['u_in_partition_out_sum'] = train.groupby(['breath_id',"u_out"])['u_in'].transform("sum")

    train = train.fillna(0) # add for consistency with how test is done - dan

    return train

train = add_feats(train)
test = add_feats(test)

6036000


,id,breath_id,R,C,time_step,u_in,u_out,pressure,last_value_u_in,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag3,u_out_lag3,u_in_lag_back2,u_out_lag_back2,u_in_lag_back3,u_out_lag_back3,u_in_lag_back10,u_out_lag_back10,u_in_first,u_out_first,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,15_in_sum,15_in_min,15_in_max,15_in_mean,15_out_std,45_in_sum,45_in_min,45_in_max,45_in_mean,45_out_std,15_out_mean
0,1,1,20,50,0.000000,0.083334,0,5.837492,4.987079,NaN,NaN,18.383041,0.0,NaN,NaN,NaN,NaN,22.509278,0.0,22.808822,0.0,26.866758,0.0,NaN,NaN,0.000000,0.083334,NaN,NaN,0.083334,0.083334,0.083334,0.083334,NaN,0.083334,0.083334,0.083334,0.083334,NaN,0.0
1,2,1,20,50,0.033652,18.383041,0,5.907794,4.987079,0.083334,0.0,22.509278,0.0,NaN,NaN,NaN,NaN,22.808822,0.0,25.355850,0.0,26.762803,0.0,0.083334,0.0,0.033652,9.585358,12.939847,1.0,18.466375,0.083334,18.383041,9.233188,12.939847,18.466375,0.083334,18.383041,9.233188,12.939847,0.0
2,3,1,20,50,0.067514,22.509278,0,7.876254,4.987079,18.383041,0.0,22.808822,0.0,0.083334,0.0,NaN,NaN,25.355850,0.0,27.259866,0.0,27.993274,0.0,12.184338,0.0,0.033862,14.229040,11.777739,1.0,40.975653,0.083334,22.509278,13.658551,11.936136,40.975653,0.083334,22.509278,13.658551,11.936136,0.0
3,4,1,20,50,0.101542,22.808822,0,11.742872,4.987079,22.509278,0.0,25.355850,0.0,18.383041,0.0,0.083334,0.0,27.259866,0.0,27.127486,0.0,26.789898,0.0,0.000000,0.0,0.034028,16.627759,10.448647,1.0,63.784476,0.083334,22.808822,15.946119,10.766279,63.784476,0.083334,22.808822,15.946119,10.766279,0.0
4,5,1,20,50,0.135756,25.355850,0,12.234987,4.987079,22.808822,0.0,27.259866,0.0,22.509278,0.0,18.383041,0.0,27.127486,0.0,26.807732,0.0,25.634070,0.0,0.000000,0.0,0.034213,18.652046,9.801731,1.0,89.140326,0.083334,25.355850,17.828065,10.229525,89.140326,0.083334,25.355850,17.828065,10.229525,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032,1.482739,1.420711,1.0,1.488497,1.0,1.353205,1.0,1.357586,1.0,1.558978,1.0,1.272663,1.0,NaN,NaN,NaN,NaN,0.033412,1.626944,2.367440,1.0,19.228896,1.113529,1.489714,1.281926,0.117841,140.943536,0.000000,14.017899,3.132079,4.277354,1.0
6035996,6035997,125749,50,10,2.537961,1.488497,1,3.869032,1.482739,1.489714,1.0,1.558978,1.0,1.420711,1.0,1.353205,1.0,1.272663,1.0,1.482739,1.0,NaN,NaN,NaN,NaN,0.033358,1.616654,2.287248,1.0,19.603865,1.137701,1.489714,1.306924,0.119329,142.432033,0.000000,14.017899,3.165156,4.258295,1.0
6035997,6035998,125749,50,10,2.571408,1.558978,1,3.798729,1.482739,1.488497,1.0,1.272663,1.0,1.489714,1.0,1.420711,1.0,1.482739,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.033447,1.612368,2.209336,1.0,19.995327,1.137701,1.558978,1.333022,0.129071,130.090426,0.000000,14.017899,2.890898,3.936430,1.0
6035998,6035999,125749,50,10,2.604744,1.272663,1,4.079938,1.482739,1.558978,1.0,1.482739,1.0,1.488497,1.0,1.489714,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033337,1.587130,2.136635,1.0,20.115881,1.137701,1.558978,1.341059,0.120468,117.345191,0.000000,12.616385,2.607671,3.557961,1.0


before OHE


,id,breath_id,R,C,time_step,u_in,u_out,pressure,last_value_u_in,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag3,u_out_lag3,u_in_lag_back2,u_out_lag_back2,u_in_lag_back3,u_out_lag_back3,u_in_lag_back10,u_out_lag_back10,u_in_first,u_out_first,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,15_in_sum,15_in_min,15_in_max,15_in_mean,15_out_std,45_in_sum,45_in_min,45_in_max,45_in_mean,45_out_std,15_out_mean,breath_id__u_in__max,breath_id__u_out__max,breath_id__u_out__mean,breath_id__u_in__mean,breath_id__u_in__min,breath_id__u_out__min,u_in_last,R_div_C,R__C,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff_1_2,u_out_diff_1_2,u_in_lagback_diff_1_2,u_out_lagback_diff_1_2,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,u_in_diff,u_out_diff,breath_id__u_in__diffmax,breath_id__u_in__diffmean
0,1,1,20,50,0.000000,0.083334,0,5.837492,4.987079,0.000000,0.0,18.383041,0.0,0.000000,0.0,0.000000,0.0,22.509278,0.0,22.808822,0.0,26.866758,0.0,0.083334,0.0,0.000000,0.083334,0.000000,0.0,0.083334,0.083334,0.083334,0.083334,0.000000,0.083334,0.083334,0.083334,0.083334,0.000000,0.0,28.313036,1,NaN,NaN,0.0,0,4.987079,0.4,20__50,0.083334,0.0,0.083334,0.0,0.083334,0.0,0.000000,0.0,-4.126236,0.0,-18.299707,0.0,-22.425944,0.0,0.0,0.0,28.229702,10.062673
1,2,1,20,50,0.033652,18.383041,0,5.907794,4.987079,0.083334,0.0,22.509278,0.0,0.000000,0.0,0.000000,0.0,22.808822,0.0,25.355850,0.0,26.762803,0.0,0.083334,0.0,0.033652,9.585358,12.939847,1.0,18.466375,0.083334,18.383041,9.233188,12.939847,18.466375,0.083334,18.383041,9.233188,12.939847,0.0,28.313036,1,0.6250,10.146007,0.0,0,4.987079,0.4,20__50,18.299707,0.0,18.383041,0.0,18.383041,0.0,0.083334,0.0,-0.299544,0.0,-4.126236,0.0,-4.425781,0.0,NaN,NaN,9.929994,-8.237035
2,3,1,20,50,0.067514,22.509278,0,7.876254,4.987079,18.383041,0.0,22.808822,0.0,0.083334,0.0,0.000000,0.0,25.355850,0.0,27.259866,0.0,27.993274,0.0,0.083334,0.0,0.033862,14.229040,11.777739,1.0,40.975653,0.083334,22.509278,13.658551,11.936136,40.975653,0.083334,22.509278,13.658551,11.936136,0.0,28.313036,1,0.6375,5.093747,0.0,0,4.987079,0.4,20__50,4.126236,0.0,22.425944,0.0,22.509278,0.0,18.299707,0.0,-2.547028,0.0,-0.299544,0.0,-2.846573,0.0,NaN,NaN,5.803758,-12.363271
3,4,1,20,50,0.101542,22.808822,0,11.742872,4.987079,22.509278,0.0,25.355850,0.0,18.383041,0.0,0.083334,0.0,27.259866,0.0,27.127486,0.0,26.789898,0.0,0.083334,0.0,0.034028,16.627759,10.448647,1.0,63.784476,0.083334,22.808822,15.946119,10.766279,63.784476,0.083334,22.808822,15.946119,10.766279,0.0,28.313036,1,0.6000,5.457599,0.0,0,4.987079,0.4,20__50,0.299544,0.0,4.425781,0.0,22.725488,0.0,4.126236,0.0,-1.904016,0.0,-2.547028,0.0,-4.451044,0.0,NaN,NaN,5.504214,-12.662816
4,5,1,20,50,0.135756,25.355850,0,12.234987,4.987079,22.808822,0.0,27.259866,0.0,22.509278,0.0,18.383041,0.0,27.127486,0.0,26.807732,0.0,25.634070,0.0,0.083334,0.0,0.034213,18.652046,9.801731,1.0,89.140326,0.083334,25.355850,17.828065,10.229525,89.140326,0.083334,25.355850,17.828065,10.229525,0.0,28.313036,1,0.6000,6.002238,0.0,0,4.987079,0.4,20__50,2.547028,0.0,2.846573,0.0,6.972809,0.0,0.299544,0.0,0.132380,0.0,-1.904016,0.0,-1.771635,0.0,NaN,NaN,2.957185,-15.209844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,6035996,125749,50,10,2.504603,1.489714,1,3.869032,1.482739,1.420711,1.0,1.488497,1.0,1.353205,1.0,1.357586,1.0,1.558978,1.0,1.272663,1.0,0.000000,0.0,6.030572,0.0,0.033412,1.626944,2.367440,1.0,19.228896,1.113529,1.489714,1.281926,0.117841,140.943536,0.000000,14.017899,3.132079,4.277354,1.0,25.504196,1,NaN,NaN,0.0,0,1.482739,5.0,50__10,0.069002,0.0,0.136509,0.0,0.132127,0.0,0.067506,0.0,-0.070481,0.0,0.001217,0.0,-0.069264,0.0,NaN,NaN,24.014482,3.421689
6035996,6035997,125749,5

4024000


,id,breath_id,R,C,time_step,u_in,u_out,last_value_u_in,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag3,u_out_lag3,u_in_lag_back2,u_out_lag_back2,u_in_lag_back3,u_out_lag_back3,u_in_lag_back10,u_out_lag_back10,u_in_first,u_out_first,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,15_in_sum,15_in_min,15_in_max,15_in_mean,15_out_std,45_in_sum,45_in_min,45_in_max,45_in_mean,45_out_std,15_out_mean
0,1,0,5,20,0.000000,0.000000,0,4.973375,NaN,NaN,7.515046,0.0,NaN,NaN,NaN,NaN,14.651675,0.0,21.230610,0.0,37.434198,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.0,0.000000,0.000000,NaN,0.0
1,2,0,5,20,0.031904,7.515046,0,4.973375,0.000000,0.0,14.651675,0.0,NaN,NaN,NaN,NaN,21.230610,0.0,26.320956,0.0,37.137143,0.0,NaN,NaN,0.031904,3.902147,5.313940,1.0,7.515046,0.000000,7.515046,3.757523,5.313940,7.515046,0.0,7.515046,3.757523,5.313940,0.0
2,3,0,5,20,0.063827,14.651675,0,4.973375,7.515046,0.0,21.230610,0.0,0.000000,0.0,NaN,NaN,26.320956,0.0,30.486938,0.0,35.937643,0.0,NaN,NaN,0.031924,7.764551,7.319352,1.0,22.166721,0.000000,14.651675,7.388907,7.326652,22.166721,0.0,14.651675,7.388907,7.326652,0.0
3,4,0,5,20,0.095751,21.230610,0,4.973375,14.651675,0.0,26.320956,0.0,7.515046,0.0,0.000000,0.0,30.486938,0.0,33.545950,0.0,34.712418,0.0,NaN,NaN,0.031924,11.529365,9.121700,1.0,43.397331,0.000000,21.230610,10.849333,9.147936,43.397331,0.0,21.230610,10.849333,9.147936,0.0
4,5,0,5,20,0.127644,26.320956,0,4.973375,21.230610,0.0,30.486938,0.0,14.651675,0.0,7.515046,0.0,33.545950,0.0,35.717600,0.0,33.132383,0.0,NaN,NaN,0.031893,14.959946,10.439697,1.0,69.718287,0.000000,26.320956,13.943657,10.518450,69.718287,0.0,26.320956,13.943657,10.518450,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,4023996,125748,20,10,2.530117,4.971245,1,4.985373,4.965959,1.0,4.975709,1.0,4.959653,1.0,4.952134,1.0,4.979468,1.0,4.982648,1.0,NaN,NaN,NaN,NaN,0.033753,4.212495,1.773687,1.0,73.183249,4.693262,4.971245,4.878883,0.087015,125.753821,0.0,4.971245,2.794529,2.152980,1.0
4023996,4023997,125748,20,10,2.563853,4.975709,1,4.985373,4.971245,1.0,4.979468,1.0,4.965959,1.0,4.959653,1.0,4.982648,1.0,4.985373,1.0,NaN,NaN,NaN,NaN,0.033736,4.269219,1.727962,1.0,73.465696,4.741088,4.975709,4.897713,0.073486,130.729530,0.0,4.975709,2.905101,2.133884,1.0
4023997,4023998,125748,20,10,2.597475,4.979468,1,4.985373,4.975709,1.0,4.982648,1.0,4.971245,1.0,4.965959,1.0,4.985373,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.033622,4.321997,1.682338,1.0,73.704075,4.781253,4.979468,4.913605,0.062087,135.708997,0.0,4.979468,3.015755,2.108773,1.0
4023998,4023999,125748,20,10,2.631134,4.982648,1,4.985373,4.979468,1.0,4.985373,1.0,4.975709,1.0,4.971245,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033659,4.371079,1.636955,1.0,73.905470,4.815235,4.982648,4.927031,0.052449,140.691645,0.0,4.982648,3.126481,2.077404,1.0


before OHE


,id,breath_id,R,C,time_step,u_in,u_out,last_value_u_in,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag3,u_out_lag3,u_in_lag_back2,u_out_lag_back2,u_in_lag_back3,u_out_lag_back3,u_in_lag_back10,u_out_lag_back10,u_in_first,u_out_first,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,15_in_sum,15_in_min,15_in_max,15_in_mean,15_out_std,45_in_sum,45_in_min,45_in_max,45_in_mean,45_out_std,15_out_mean,breath_id__u_in__max,breath_id__u_out__max,breath_id__u_out__mean,breath_id__u_in__mean,breath_id__u_in__min,breath_id__u_out__min,u_in_last,R_div_C,R__C,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff_1_2,u_out_diff_1_2,u_in_lagback_diff_1_2,u_out_lagback_diff_1_2,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,u_in_diff,u_out_diff,breath_id__u_in__diffmax,breath_id__u_in__diffmean
0,1,0,5,20,0.000000,0.000000,0,4.973375,0.000000,0.0,7.515046,0.0,0.000000,0.0,0.000000,0.0,14.651675,0.0,21.230610,0.0,37.434198,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,37.542219,1,0.6125,9.327338,0.0,0,4.973375,0.25,5__20,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,-7.136630,0.0,-7.515046,0.0,-14.651675,0.0,0.0,0.0,37.542219,9.327338
1,2,0,5,20,0.031904,7.515046,0,4.973375,0.000000,0.0,14.651675,0.0,0.000000,0.0,0.000000,0.0,21.230610,0.0,26.320956,0.0,37.137143,0.0,0.000000,0.0,0.031904,3.902147,5.313940,1.0,7.515046,0.000000,7.515046,3.757523,5.313940,7.515046,0.0,7.515046,3.757523,5.313940,0.0,37.542219,1,NaN,NaN,0.0,0,4.973375,0.25,5__20,7.515046,0.0,7.515046,0.0,7.515046,0.0,0.000000,0.0,-6.578935,0.0,-7.136630,0.0,-13.715564,0.0,NaN,NaN,30.027173,1.812292
2,3,0,5,20,0.063827,14.651675,0,4.973375,7.515046,0.0,21.230610,0.0,0.000000,0.0,0.000000,0.0,26.320956,0.0,30.486938,0.0,35.937643,0.0,0.000000,0.0,0.031924,7.764551,7.319352,1.0,22.166721,0.000000,14.651675,7.388907,7.326652,22.166721,0.0,14.651675,7.388907,7.326652,0.0,37.542219,1,NaN,NaN,0.0,0,4.973375,0.25,5__20,7.136630,0.0,14.651675,0.0,14.651675,0.0,7.515046,0.0,-5.090346,0.0,-6.578935,0.0,-11.669281,0.0,NaN,NaN,22.890543,-5.324338
3,4,0,5,20,0.095751,21.230610,0,4.973375,14.651675,0.0,26.320956,0.0,7.515046,0.0,0.000000,0.0,30.486938,0.0,33.545950,0.0,34.712418,0.0,0.000000,0.0,0.031924,11.529365,9.121700,1.0,43.397331,0.000000,21.230610,10.849333,9.147936,43.397331,0.0,21.230610,10.849333,9.147936,0.0,37.542219,1,NaN,NaN,0.0,0,4.973375,0.25,5__20,6.578935,0.0,13.715564,0.0,21.230610,0.0,7.136630,0.0,-4.165982,0.0,-5.090346,0.0,-9.256328,0.0,NaN,NaN,16.311609,-11.903272
4,5,0,5,20,0.127644,26.320956,0,4.973375,21.230610,0.0,30.486938,0.0,14.651675,0.0,7.515046,0.0,33.545950,0.0,35.717600,0.0,33.132383,0.0,0.000000,0.0,0.031893,14.959946,10.439697,1.0,69.718287,0.000000,26.320956,13.943657,10.518450,69.718287,0.0,26.320956,13.943657,10.518450,0.0,37.542219,1,NaN,NaN,0.0,0,4.973375,0.25,5__20,5.090346,0.0,11.669281,0.0,18.805911,0.0,6.578935,0.0,-3.059012,0.0,-4.165982,0.0,-7.224994,0.0,NaN,NaN,11.221263,-16.993619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,4023996,125748,20,10,2.530117,4.971245,1,4.985373,4.965959,1.0,4.975709,1.0,4.959653,1.0,4.952134,1.0,4.979468,1.0,4.982648,1.0,0.000000,0.0,9.985853,0.0,0.033753,4.212495,1.773687,1.0,73.183249,4.693262,4.971245,4.878883,0.087015,125.753821,0.0,4.971245,2.794529,2.152980,1.0,35.151391,1,NaN,NaN,0.0,0,4.985373,2.00,20__10,0.005286,0.0,0.011593,0.0,0.019111,0.0,0.006307,0.0,-0.003759,0.0,-0.004463,0.0,-0.008222,0.0,NaN,NaN,30.180146,-2.288270
4023996,4023997,125748,20,10,2.563853,4.975709,1,4.985373,4.971245,1.0,4.979468,1.0,4.965959,1.0,4.959653,1.0,4.982648,1.0,4.985373,1.0,0.000000,0.0,9.985853,0.0,0.03373

In [7]:
train.head()

,id,breath_id,time_step,u_in,u_out,pressure,last_value_u_in,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag3,u_out_lag3,u_in_lag_back2,u_out_lag_back2,u_in_lag_back3,u_out_lag_back3,u_in_lag_back10,u_out_lag_back10,u_in_first,u_out_first,time_step_diff,ewm_u_in_mean,ewm_u_in_std,ewm_u_in_corr,15_in_sum,15_in_min,15_in_max,15_in_mean,15_out_std,45_in_sum,45_in_min,45_in_max,45_in_mean,45_out_std,15_out_mean,breath_id__u_in__max,breath_id__u_out__max,...,breath_id__u_in__min,breath_id__u_out__min,u_in_last,R_div_C,u_in_diff1,u_out_diff1,u_in_diff2,u_out_diff2,u_in_diff3,u_out_diff3,u_in_diff_1_2,u_out_diff_1_2,u_in_lagback_diff_1_2,u_out_lagback_diff_1_2,u_in_lagback_diff1,u_out_lagback_diff1,u_in_lagback_diff2,u_out_lagback_diff2,u_in_diff,u_out_diff,breath_id__u_in__diffmax,breath_id__u_in__diffmean,R_5,R_20,R_50,C_10,C_20,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50,u_in_cumsum,time_step_cumsum,u_in_partition_out_sum
0,1,1,0.000000,0.083334,0,5.837492,4.987079,0.000000,0.0,18.383041,0.0,0.000000,0.0,0.000000,0.0,22.509278,0.0,22.808822,0.0,26.866758,0.0,0.083334,0.0,0.000000,0.083334,0.000000,0.0,0.083334,0.083334,0.083334,0.083334,0.000000,0.083334,0.083334,0.083334,0.083334,0.000000,0.0,28.313036,1,...,0.0,0,4.987079,0.4,0.083334,0.0,0.083334,0.0,0.083334,0.0,0.000000,0.0,-4.126236,0.0,-18.299707,0.0,-22.425944,0.0,0.0,0.0,28.229702,10.062673,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0.083334,0.000000,663.594727
1,2,1,0.033652,18.383041,0,5.907794,4.987079,0.083334,0.0,22.509278,0.0,0.000000,0.0,0.000000,0.0,22.808822,0.0,25.355850,0.0,26.762803,0.0,0.083334,0.0,0.033652,9.585358,12.939847,1.0,18.466375,0.083334,18.383041,9.233188,12.939847,18.466375,0.083334,18.383041,9.233188,12.939847,0.0,28.313036,1,...,0.0,0,4.987079,0.4,18.299707,0.0,18.383041,0.0,18.383041,0.0,0.083334,0.0,-0.299544,0.0,-4.126236,0.0,-4.425781,0.0,0.0,0.0,9.929994,-8.237035,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,18.466375,0.033652,663.594727
2,3,1,0.067514,22.509278,0,7.876254,4.987079,18.383041,0.0,22.808822,0.0,0.083334,0.0,0.000000,0.0,25.355850,0.0,27.259866,0.0,27.993274,0.0,0.083334,0.0,0.033862,14.229040,11.777739,1.0,40.975653,0.083334,22.509278,13.658551,11.936136,40.975653,0.083334,22.509278,13.658551,11.936136,0.0,28.313036,1,...,0.0,0,4.987079,0.4,4.126236,0.0,22.425944,0.0,22.509278,0.0,18.299707,0.0,-2.547028,0.0,-0.299544,0.0,-2.846573,0.0,0.0,0.0,5.803758,-12.363271,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,40.975653,0.101167,663.594727
3,4,1,0.101542,22.808822,0,11.742872,4.987079,22.509278,0.0,25.355850,0.0,18.383041,0.0,0.083334,0.0,27.259866,0.0,27.127486,0.0,26.789898,0.0,0.083334,0.0,0.034028,16.627759,10.448647,1.0,63.784476,0.083334,22.808822,15.946119,10.766279,63.784476,0.083334,22.808822,15.946119,10.766279,0.0,28.313036,1,...,0.0,0,4.987079,0.4,0.299544,0.0,4.425781,0.0,22.725488,0.0,4.126236,0.0,-1.904016,0.0,-2.547028,0.0,-4.451044,0.0,0.0,0.0,5.504214,-12.662816,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,63.784476,0.202709,663.594727
4,5,1,0.135756,25.355850,0,12.234987,4.987079,22.808822,0.0,27.259866,0.0,22.509278,0.0,18.383041,0.0,27.127486,0.0,26.807732,0.0,25.634070,0.0,0.083334,0.0,0.034213,18.652046,9.801731,1.0,89.140326,0.083334,25.355850,17.828065,10.229525,89.140326,0.083334,25.355850,17.828065,10.229525,0.0,28.313036,1,...,0.0,0,4.987079,0.4,2.547028,0.0,2.846573,0.0,6.972809,0.0,0.299544,0.0,0.132380,0.0,-1.904016,0.0,-1.771635,0.0,0.0,0.0,2.957185,-15.209844,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,89.140326,0.338464,663.594727


In [8]:
train.shape

(6036000, 82)

In [9]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
u_outs = train[['u_out']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

In [10]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [11]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [12]:
def GBVPP_loss(y_true, y_pred, cols = 80):
    u_out = y_true[:, cols: ]
    y = y_true[:, :cols ]

    w = 1 - u_out
    mae = w * tf.abs(y - y_pred)
    return tf.reduce_sum(mae, axis=-1) / tf.reduce_sum(w, axis=-1)

In [13]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K

class Attention(Layer):
    
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention,self).__init__()
        
    def build(self, input_shape):
        
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
        super(Attention,self).build(input_shape)
        
    def call(self, x):
        
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return K.sum(output, axis=1)

    def get_config(self):
        return super(Attention, self).get_config()

In [20]:
def get_cnn_model(train):
  inputs = keras.layers.Input(shape=train.shape[-2:])
  X = inputs

  X = keras.layers.Conv1D(filters=2048, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.Conv1D(filters=2048, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.AveragePooling1D(pool_size=2)(X)

  X = Attention(return_sequences=True)(X)

  X = keras.layers.Conv1D(filters=1024, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.Conv1D(filters=1024, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.AveragePooling1D(pool_size=2)(X)

  X = Attention(return_sequences=True)(X)


  X = keras.layers.Conv1D(filters=512, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.Conv1D(filters=512, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.AveragePooling1D(pool_size=2)(X)

  X = Attention(return_sequences=True)(X)

  X = keras.layers.Conv1D(filters=256, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.Conv1D(filters=256, kernel_size=3, activation='relu')(X)
  X = keras.layers.BatchNormalization()(X)
  X = keras.layers.AveragePooling1D(pool_size=2)(X)

  X = Attention(return_sequences=True)(X)

  X = keras.layers.Flatten()(X) #256

  input_X = keras.layers.Flatten()(inputs)
  input_X = keras.layers.Dense(512, activation='selu', kernel_initializer='lecun_normal')(input_X)
  input_X = keras.layers.BatchNormalization()(input_X)
  input_X = keras.layers.Dense(256, activation='selu', kernel_initializer='lecun_normal')(input_X)
  input_X = keras.layers.BatchNormalization()(input_X) #256

  X = keras.layers.Concatenate()([X, input_X])

  X = keras.layers.Dense(256, activation='selu', kernel_initializer='lecun_normal')(X)
  X = keras.layers.BatchNormalization()(X)
  outputs = keras.layers.Dense(80)(X)
  model  = keras.Model(inputs=inputs, outputs=outputs)
  model.summary()
  return model


In [ ]:
EPOCH = 1000
BATCH_SIZE = 1024
NUM_FOLDS = 10

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
    test_preds = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        u_out_train, u_out_valid = u_outs[train_idx], u_outs[test_idx]
        
        model = get_cnn_model(train)
        model.compile(optimizer='adam', 
                      # loss=GBVPP_loss,
                      loss="mae"
                      )

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.75, patience=10, verbose=1)
        es = EarlyStopping(monitor="val_loss", patience=50, verbose=1, mode="min", restore_best_weights=True)
        checkpoint_filepath = f"folds{fold}.hdf5"
        sv = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
            save_weights_only=False, mode='auto', save_freq='epoch',
            options=None)

        history = model.fit(X_train,
                            y_train,
                            # validation_data=(X_valid, np.append(y_valid, u_out_valid, axis =1)), 
                            validation_data=(X_valid, y_valid), 
                            epochs=EPOCH, 
                            batch_size=1024, 
                            callbacks=[lr,
                                      #  es,
                                       sv])
    
        test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.21.138.154:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.21.138.154:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Streaming output truncated to the last 5000 lines.
Epoch 00843: val_loss did not improve from 0.30604
Epoch 844/1000
67/67 [==============================] - 6s 85ms/step - loss: 0.4025 - val_loss: 0.3088

Epoch 00844: ReduceLROnPlateau reducing learning rate to 7.568015224918012e-12.

Epoch 00844: val_loss did not improve from 0.30604
Epoch 845/1000
67/67 [==============================] - 6s 86ms/step - loss: 0.4019 - val_loss: 0.3094

Epoch 00845: val_loss did not improve from 0.30604
Epoch 846/1000
67/67 [==============================] - 6s 86ms/step - loss: 0.3988 - val_loss: 0.3084

Epoch 00846: val_loss did not improve from 0.30604
Epoch 847/1000
67/67 [==============================] - 6s 86ms/step - loss: 0.4086 - val_loss: 0.3084

Epoch 00847: val_loss did not improve from 0.30604
Epoch 848/1000
67/67 [==============================] - 6s 86ms/step - loss: 0.4034 - val_loss: 0.3090

Epoch 00848: val_loss did not improve from 0.30604
Epoch 849/1000
67/67 [====================

In [ ]:
PRESSURE_STEP = 0.07030214545120961
PRESSURE_MIN = -1.8957442945646408
PRESSURE_MAX = 64.82099173863948

In [ ]:
# ENSEMBLE FOLDS WITH MEAN
submission["pressure"] = sum(test_preds)/NUM_FOLDS
submission.to_csv('submission_mean.csv', index=False)

In [ ]:
# ENSEMBLE FOLDS WITH MEDIAN
submission["pressure"] = np.median(np.vstack(test_preds),axis=0)
submission.to_csv('submission_median.csv', index=False)

In [ ]:
# ENSEMBLE FOLDS WITH MEDIAN AND ROUND PREDICTIONS
submission["pressure"] =\
    np.round( (submission.pressure - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
submission.pressure = np.clip(submission.pressure, PRESSURE_MIN, PRESSURE_MAX)
submission.to_csv('submission_median_round.csv', index=False)

In [ ]:
!zip cnn_models.zip *.hdf5

In [ ]:
!zip submission.zip *.csv

In [ ]:
with open('/gdrive/MyDrive/ventilator-pressure-prediction/submission.zip', 'wb') as f:
  f.write(open('submission.zip', 'rb').read())

In [ ]:
with open('/gdrive/MyDrive/ventilator-pressure-prediction/cnn_models.zip', 'wb') as f:
  f.write(open('cnn_models.zip', 'rb').read())

In [ ]:
!unzip -t /gdrive/MyDrive/ventilator-pressure-prediction/cnn_models.zip